##### Copyright 2020 The TensorFlow Authors.

In [8]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Keras Tuner 소개

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/keras/keras_tuner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">View on TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a></td>
</table>

## 개요

Keras Tuner는 TensorFlow 프로그램에 대한 최적의 하이퍼파라미터 세트를 선택하는 데 도움을 주는 라이브러리입니다. 머신러닝(ML) 애플리케이션에 대한 올바른 하이퍼파라미터 세트를 선택하는 과정을 *하이퍼파라미터 조정* 또는 *하이퍼튜닝*이라고 합니다.

하이퍼파라미터는 훈련 프로세스 및 ML 모델의 토폴로지를 제어하는 변수입니다. 이러한 변수는 훈련 과정에서 일정하게 유지되며 ML 프로그램의 성능에 직접적으로 영향을 미칩니다. 하이퍼파라미터에는 두 가지 유형이 있습니다.

1. 숨겨진 레이어의 수 및 너비와 같이 모델 선택에 영향을 미치는 **모델 하이퍼파라미터**
2. SGD(Stochastic Gradient Descent)의 학습률 및 KNN(k Nearest Neighbors) 분류자의 최근접 이웃 수와 같은 학습 알고리즘의 속도와 품질에 영향을 주는 **알고리즘 하이퍼파라미터**

이 튜토리얼에서는 Keras Tuner를 사용하여 이미지 분류 애플리케이션에 하이퍼튜닝을 수행합니다.

## 설정

In [9]:
import tensorflow as tf
from tensorflow import keras

import IPython

Keras Tuner를 설치하고 가져옵니다.

In [10]:
!pip install -U keras-tuner
import kerastuner as kt

Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.7/dist-packages (1.0.2)


## 데이터세트 다운로드 및 준비하기

이 튜토리얼에서는 Keras Tuner를 사용하여 [Fashion MNIST 데이터세트](https://github.com/zalandoresearch/fashion-mnist)에서 의류 이미지를 분류하는 머신러닝 모델에 가장 적합한 하이퍼파라미터를 찾습니다. 

데이터를 로드합니다.

In [11]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [12]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## 모델 정의하기

하이퍼튜닝을 위한 모델을 빌드할 때는 모델 아키텍처와 더불어 하이퍼파라미터 검색 공간도 정의합니다. 하이퍼튜닝을 위해 설정하는 모델을 *하이퍼 모델*이라고 합니다.

두 가지 접근 방식을 통해 하이퍼 모델을 정의할 수 있습니다.

- 모델 빌더 함수 사용
- Keras Tuner API의 `HyperModel` 클래스를 하위 클래스화

또한 두 개의 사전 정의된 `HyperModel` - 클래스인 [HyperXception](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperxception-class)과 [HyperResNet](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperresnet-class)을 컴퓨터 비전 애플리케이션에 사용할 수 있습니다.

이 튜토리얼에서는 모델 빌더 함수를 사용하여 이미지 분류 모델을 정의합니다. 모델 빌더 함수는 컴파일된 모델을 반환하고 인라인으로 정의한 하이퍼파라미터를 사용하여 모델을 하이퍼튜닝합니다.

In [13]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model

## 튜너를 인스턴스화하고 하이퍼튜닝 수행하기

튜너를 인스턴스화하여 하이퍼튜닝을 수행합니다. Keras Tuner에는 `RandomSearch`, `Hyperband`, `BayesianOptimization` 및 `Sklearn`의 네 가지 튜너가 있습니다. 이 튜토리얼에서는 [Random Search](https://arxiv.org/pdf/1603.06560.pdf) 튜너를 사용합니다.

## 과제

In [14]:
# 이곳에 Random Search를 수해하는 tuner 오브젝트를 코딩하십시오.
tuner =  kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

하이퍼파라미터 검색을 실행하기 전에 훈련 단계가 끝날 때마다 훈련 결과를 지우도록 콜백을 정의합니다.

In [15]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

하이퍼파라미터 검색을 실행합니다. 검색 메서드의 인수는 위의 콜백 외에 `tf.keras.model.fit`에 사용되는 인수와 같습니다.

In [16]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 22s]
val_accuracy: 0.8719000220298767

Best val_accuracy So Far: 0.8806999921798706
Total elapsed time: 00h 12m 23s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 512 and the optimal learning rate for the optimizer
is 0.001.



이 튜토리얼을 마치려면 검색에서 최적의 하이퍼파라미터로 모델을 재훈련합니다.

In [17]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.4738 - accuracy: 0.8300 - val_loss: 0.4030 - val_accuracy: 0.8526
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3591 - accuracy: 0.8688 - val_loss: 0.3731 - val_accuracy: 0.8644
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3247 - accuracy: 0.8800 - val_loss: 0.3448 - val_accuracy: 0.8766
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2995 - accuracy: 0.8895 - val_loss: 0.3541 - val_accuracy: 0.8708
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2810 - accuracy: 0.8954 - val_loss: 0.3570 - val_accuracy: 0.8736
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2661 - accuracy: 0.9003 - val_loss: 0.3369 - val_accuracy: 0.8783
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2531 - accuracy: 0.9053 - val_loss: 0.3226 - val_accuracy:

`my_dir/intro_to_kt` 디렉토리에는 하이퍼파라미터 검색 중에 실행되는 모든 시험(모델 구성)에 대한 상세 로그와 체크포인트가 들어 있습니다. 하이퍼파라미터 검색을 다시 실행하면 Keras Tuner가 이러한 로그의 기존 상태를 사용하여 검색을 재개합니다. 이 동작을 비활성화하려면 튜너를 인스턴스화하는 동안 추가 `overwrite = True` 인수를 전달합니다.

## 요약

이 튜토리얼에서는 Keras Tuner를 사용하여 모델의 하이퍼파라미터를 조정하는 방법을 배웠습니다. Keras Tuner에 대한 자세한 내용은 다음 추가 자료를 확인하세요.

- [TensorFlow 블로그의 Keras Tuner](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
- [Keras Tuner 웹 사이트](https://keras-team.github.io/keras-tuner/)

모델 하이퍼파라미터를 능동적으로 조정하기 위한 TensorBoard의 [HParams Dashboard](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams)도 확인해 보세요.